# 🎯 GitHub மாடல்களுடன் திட்டமிடல் மற்றும் வடிவமைப்பு முறை (.NET)

## 📋 கற்றல் நோக்கங்கள்

இந்த நோட்டுப் புத்தகம் Microsoft Agent Framework-ஐ .NET-இல் பயன்படுத்தி புத்திசாலி முகவர்களை உருவாக்குவதற்கான நிறுவன தரமான திட்டமிடல் மற்றும் வடிவமைப்பு முறைகளை விளக்குகிறது. நீங்கள் சிக்கலான பிரச்சினைகளை பிரிக்கவும், பல படிகள் கொண்ட தீர்வுகளை திட்டமிடவும், .NET நிறுவன அம்சங்களுடன் சிக்கலான வேலைப்பாடுகளை செயல்படுத்தவும் முகவர்களை உருவாக்க கற்றுக்கொள்வீர்கள்.

## ⚙️ முன் தேவைகள் மற்றும் அமைப்பு

**உருவாக்க சூழல்:**
- .NET 9.0 SDK அல்லது அதற்கு மேல்
- Visual Studio 2022 அல்லது C# நீட்டிப்புடன் VS Code
- GitHub மாடல்களின் API அணுகல்

**தேவையான சார்புகள்:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**சூழல் கட்டமைப்பு (.env கோப்பு):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// #r "nuget: OpenAI, 2.4.0"

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0/Microsoft.Agents.AI.OpenAI.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [5]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;


In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
System.Environment.GetEnvironmentVariable("GITHUB_ENDPOINT");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:


var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string AGENT_NAME ="TravelPlanAgent";

const string AGENT_INSTRUCTIONS = @"You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general request";

In [15]:

    public class Plan
    {
        [JsonPropertyName("assigned_agent")]
        public string? Assigned_agent { get; set; }

        [JsonPropertyName("task_details")]
        public string? Task_details { get; set; }
    }

In [16]:

    public class TravelPlan
    {
        [JsonPropertyName("main_task")]
        public string? Main_task { get; set; }

        [JsonPropertyName("subtasks")]
        public IList<Plan> Subtasks { get; set; }

    }

In [17]:

ChatClientAgentOptions agentOptions = new(name: AGENT_NAME, instructions: AGENT_INSTRUCTIONS)
{
    ChatOptions = new()
    {
        ResponseFormat = ChatResponseFormatJson.ForJsonSchema(
            schema: AIJsonUtilities.CreateJsonSchema(typeof(TravelPlan)),
            schemaName: "TravelPlan",
            schemaDescription: "Travel Plan with main_task and subtasks")
    }
};

In [18]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    agentOptions);

In [19]:

Console.WriteLine(await agent.RunAsync("Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne"));

{"main_task":"Plan a family trip from Singapore to Melbourne","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Find and book flights from Singapore to Melbourne for a family of 4."},{"assigned_agent":"HotelBooking","task_details":"Find and book a family-friendly hotel in Melbourne."},{"assigned_agent":"CarRental","task_details":"Arrange car rental in Melbourne for local transportation."},{"assigned_agent":"ActivitiesBooking","task_details":"Book family-friendly activities and attractions in Melbourne."},{"assigned_agent":"DestinationInfo","task_details":"Provide information about Melbourne, including dining and family-friendly areas."}]}



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
